In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10
import tensorflow_datasets as tfds
import tensorflow as tf

## Feed-Forward Neural Network for Digit Classification

In [ ]:
print("[INFO] accessing MNIST...")
## The MNIST digit classification dataset is popular enough that it's available within the keras package!
((trainX, trainY), (testX, testY)) = mnist.load_data()

In [ ]:
# reshape data into 1D array of pixel values
trainX = trainX.reshape((trainX.shape[0], 28 * 28 * 1))
testX = testX.reshape((testX.shape[0], 28 * 28 * 1))

#normalize between 0 and 1
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0


In [ ]:
# create labels for each of the 10 digits we want to classify
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# Seqeuential <== defining a feed-forward neural network; sequentially add layers on top
model = Sequential()
## Dense layers <== fully connected neural network layer
model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))


In [ ]:
# train the model using SGD
print("[INFO] training network...")
sgd = SGD(0.01)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])


H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=100, batch_size=128)

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1),
	target_names=[str(x) for x in lb.classes_]))


## RNNs with IMDb Movie Reviews

In [ ]:
## Binary Classification! Many-to-One RNN

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

# shuffle the data, update model parameters based on batch learning 
batch_size = 32
train_dataset = train_dataset.shuffle(10000) 
train_dataset = train_dataset.batch(batch_size) 
test_dataset = test_dataset.batch(batch_size)


In [ ]:
## retain at most 1000 words in our vocabulary
VOCAB_SIZE = 1000
## create an encoder that vectorizes raw text into word IDs.
encoder = TextVectorization(
    max_tokens=VOCAB_SIZE, standardize='lower_and_strip_punctuation')
## fit encoder to strings/tokens that are in text; each word in dataset gets a unique ID
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = Sequential([
    encoder,
    ## embed token IDs into embeddings of size 64  
    Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    #a RNN layer that creates feedback loops between input sequences
    SimpleRNN(64),
    ## Classification Head
    Dense(64, activation='relu'),
    Dense(1)
])


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              ## Adam optimizer is another popular optimizer used for minimizing loss
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [ ]:
H = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


## CNNs with CIFAR dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
model = Sequential()
## Convolutional Filter with size 3x3
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
## Max Pooling using 2x2 patch
model.add(MaxPooling2D((2, 2)))
## Repeat 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# Final classificaiton head; features extracted from images are flattened into a feed-forward neural network
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10))


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

H = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))
